<a href="https://colab.research.google.com/github/carlosverenka/SDW_VrNk/blob/main/SDW2023_VrNk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 Etapa: Realizar o Extract do arquivo CSV

In [ ]:
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

Importamos o pandas para manipulação de informações das planilhas, retornando um dataframe

In [ ]:
import pandas as pd
df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print (user_ids)


[4990, 4991, 4992]


Recuperando o usuário a partir do GET do Swagger UI

In [ ]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None
#Vai retornar um valor se a resposta obtida na pesquisa no banco de dados (SWAGGER) for igual a 200 (Operation Successful)

users = [user for id in user_ids if(user := get_user(id)) is not None]

print(json.dumps(users, indent=2))
#biblioteca json para imprimir os usuários formatados

**TRANFORM:**

Integração com a biblioteca da openAI

In [ ]:
!pip install openai

Chave da API: https://platform.openai.com/account/api-keys

In [ ]:
#@title
openai_api_key = 'key'

In [ ]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em marketing bancário"
      },
      {"role": "user",
       "content": f"Crie uma mensagem para {user['name']} sobre  as vantagens de realizar investimentos (máximo 150 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip ('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "description": news
  })

Olá Carlos! Investir multiplica sua riqueza, garante futuro financeiro seguro e ajuda a bater a inflação. Vamos começar?
Oi Jhon! Invista e veja seu dinheiro crescer! Garanta uma renda estável, segurança financeira e tranquilidade para o futuro. #InvestimentoBancario
Oi Max! Invista e multiplique seu dinheiro com segurança. Construa seu futuro, liberdade financeira e realize sonhos. Vamos começar?


**LOAD**

In [ ]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user (user)
  print(f"User {user['name']} upadeted? {success}!")

User Carlos upadeted? True!
User Jhon upadeted? True!
User Max upadeted? True!
